In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

train = pd.read_csv("train_modified.csv")
test = pd.read_csv("test_modified.csv")

In [77]:
y = train['Product_Supermarket_Sales']
X = train.drop(['Product_Supermarket_Sales','Product_Identifier','Supermarket_Identifier','Product_Supermarket_Identifier'],axis=1)

Xtest = test.drop(['Product_Identifier','Supermarket_Identifier','Product_Supermarket_Identifier'],axis=1)

In [249]:
Xtest.head()

,Product_Price,Product_Shelf_Visibility,Product_Weight,Product_Visibility_MeanRatio,Supermarket_Years,Product_Fat_Content_0,Product_Fat_Content_1,Product_Fat_Content_2,Supermarket_Location_Type_0,Supermarket_Location_Type_1,...,Supermarket_0,Supermarket_1,Supermarket_2,Supermarket_3,Supermarket_4,Supermarket_5,Supermarket_6,Supermarket_7,Supermarket_8,Supermarket_9
0,459.98,0.214125,8.270,1.589432,11,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1,464.98,0.127821,8.270,0.948808,22,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
2,477.38,0.082171,7.390,1.000542,22,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,472.63,0.011791,6.115,1.473415,11,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
4,473.13,0.007038,6.115,0.879552,22,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [282]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [320]:
from xgboost.sklearn import XGBRegressor
XGB = XGBRegressor(learning_rate = 0.1, max_depth=3, cv=15, gamma=3, colsample_bytree=1)

XGB.fit(X_train, y_train)
prediction = XGB.predict(X_test)

In [321]:
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(y_test,prediction))
print('RMSE = ' + str(rmse))

RMSE = 2899.7002780075704


In [322]:
result = XGB.predict(Xtest)
print(result)

[ 1807.8488  8385.343   7579.04   ... 10470.383   2105.835   8630.756 ]


In [323]:
xg_reg = pd.DataFrame({'Product_Supermarket_Identifier':test['Product_Supermarket_Identifier'],
                                  'Product_Supermarket_Sales': result},
                                 columns=['Product_Supermarket_Identifier','Product_Supermarket_Sales'])

#to csv
xg_reg.to_csv('xg_pred8.csv',index=False)

In [50]:
#Import libraries:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

target = 'Product_Supermarket_Sales'

In [62]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
                          metrics='auc', early_stopping_rounds=early_stopping_rounds, show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Product_Supermarket_Sales'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain['Product_Supermarket_Sales'].values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Product_Supermarket_Sales'], dtrain_predprob))
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [64]:
#Choose all predictors except target & IDcols

predictors = [x for x in X.columns if x not in [target]]

xgb1 = XGBRegressor(learning_rate =0.1, n_estimators=1000, max_depth=5, min_child_weight=1, gamma=0, subsample=0.8,
                    colsample_bytree=0.8, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)

modelfit(xgb1, X, y)

KeyError: '[  709.08  6381.69  6381.69 ...  5685.93 11778.   17870.07] not in index'

In [256]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

reg_pred = regressor.predict(X_test)

In [257]:
reg_rmse = sqrt(mean_squared_error(y_test,reg_pred))
print('RMSE = ' + str(reg_rmse))

RMSE = 2927.0924204241746


In [45]:
lr_pred = regressor.predict(Xtest)
lr_reg = pd.DataFrame({'Product_Supermarket_Identifier':test['Product_Supermarket_Identifier'],
                                  'Product_Supermarket_Sales': lr_pred},
                                 columns=['Product_Supermarket_Identifier','Product_Supermarket_Sales'])

#to csv
lr_reg.to_csv('lr_reg.csv',index=False)

In [258]:
from sklearn.svm import SVR
svr = SVR()
svr.fit(X_train, y_train)

svr_pred = svr.predict(X_test)

In [259]:
svr_rmse = sqrt(mean_squared_error(y_test,svr_pred))
print('RMSE = ' + str(svr_rmse))

RMSE = 4556.223937999994


In [46]:
svr_pred = svr.predict(Xtest)
svr_reg = pd.DataFrame({'Product_Supermarket_Identifier':test['Product_Supermarket_Identifier'],
                                  'Product_Supermarket_Sales': svr_pred},
                                 columns=['Product_Supermarket_Identifier','Product_Supermarket_Sales'])

#to csv
svr_reg.to_csv('svr_reg1.csv',index=False)

In [28]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

In [29]:
tree = DecisionTreeRegressor(criterion='mse', max_depth=10)
tree.fit(X_train, y_train)

tree_pred = tree.predict(X_test)

In [30]:
tree_rmse = sqrt(mean_squared_error(y_test,tree_pred))
print('RMSE = ' + str(tree_rmse))

RMSE = 3373.981645885302


In [31]:
cross_val_score(XGB, X, y, cv=5, scoring='r2')

array([0.54823716, 0.53578568, 0.57518427, 0.58677662, 0.55037644])